# YES24 국내도서 종합 베스트셀러 정보 특정 페이지 

```
아래의 주소를 참조하여 YES24 종합 베스트셀러의 데이타를 데이타프레임으로 가공하여  csv 파일로 저장하여라. 
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1

저장되는 요소는? 순위, 책제목, 저자, 출판사, 발행일, 가격, 할인가, URL

```

In [4]:
import pandas as pd
import numpy as np
import os
import requests
from bs4 import BeautifulSoup

# 1) 한페이지의 책 정보

In [5]:
# response 객체 => html 페이지

url = 'http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1'
res = requests.get(url)
html_str = res.text
res

<Response [200]>

In [6]:
# soup 객체화 => 파싱
soup = BeautifulSoup(html_str, 'html.parser')
# soup

In [7]:
# 필터링 => 반복되는 요소의 선택자를 추출 
# td.goodsTxtInfo => td 태그에 적용된 goodsTxtInfo 클래스 
# td_list = soup.select('td.goodsTxtInfo')
td_list = soup.select('.goodsTxtInfo')
len(td_list)

20

In [8]:
# 책 한권의 정보 확인 
base_url = 'http://yes24.com'

title = td_list[0].select_one('a').text
# url = base_url + td_list[0].select_one('a').get('href')
url = base_url + td_list[0].select_one('a')['href']

# temp = td_list[0].select_one('div.aupu').text
# 김정환 저 | 부케이 | 2021년 12월
temp = td_list[0].select_one('div.aupu').text.split('| ')
# print(temp)
# print(temp[0]) # 저자 
# print(temp[1]) #  출판사
# print(temp[2]) #  발행일
# print('='*50)
writer = temp[0].strip().split('\r\n')[0].strip()
publisher = temp[1].strip()
published_date = temp[2].strip()

# 가격, 할인가 
'''
<p>18,000원 →<span class="priceB">16,200원</span>(<span class="price">10%</span> 할인) | YES포인트 <span class="price">900원</span>(<span class="price">5%</span> 지급)</p>
'''
# print(td_list[0].select('p')[1])
price = td_list[0].select('p')[1].text.split()[0]
discount_price = td_list[0].select_one('span.priceB').text

print('='*50)
print('책제목 => ', title)
print('URL => ', url)
print('저자 => ', writer)
print('출판사 => ', publisher)
print('출판일 => ', published_date)
print('가격 => ', price)
print('할인가 => ', discount_price)

책제목 =>  문재인의 위로
URL =>  http://yes24.com/Product/Goods/109303129
저자 =>  문재인 저
출판사 =>  더휴먼
출판일 =>  2022년 05월
가격 =>  15,800원
할인가 =>  14,220원


In [11]:
# 2차원 리스트로 저장 후 데이타프레임으로 변경 
base_url = 'http://yes24.com'
book_list = []
rank = 0
for td in td_list:
    rank += 1
    title = td.select_one('a').text
    url = base_url + td.select_one('a').get('href')
    temp = td.select_one('div.aupu').text.split('| ')
    writer = temp[0].strip().split('\r\n')[0].strip()
    publisher = temp[1].strip()
    published_date = temp[2].strip()
    price = td.select('p')[1].text.split()[0]
    discount_price = td.select_one('span.priceB').text
    # 2차원 리스트로 추가 
    book_list.append([rank, title, writer, publisher, published_date, price, discount_price, url])

df_book = pd.DataFrame(book_list, 
                       columns=['순위', '제목','저자','출판사','발행일', '가격', '할인가', 'URL'])    
df_book.head()


,순위,제목,저자,출판사,발행일,가격,할인가,URL
0,1,문재인의 위로,문재인 저,더휴먼,2022년 05월,"15,800원","14,220원",http://yes24.com/Product/Goods/109303129
1,2,5년 후 나에게 Q&A a day (2022 메리골드 Edition),포터 스타일 저/정지현 역,토네이도,2021년 11월,"16,800원","15,120원",http://yes24.com/Product/Goods/22793441
2,3,불편한 편의점,김호연 저,나무옆의자,2021년 04월,"14,000원","12,600원",http://yes24.com/Product/Goods/99308021
3,4,작별인사,김영하 저,복복서가,2022년 05월,"14,000원","12,600원",http://yes24.com/Product/Goods/108887930
4,5,대통령 문재인 명연설 100,편집부 편,더휴먼,2022년 05월,"22,000원","19,800원",http://yes24.com/Product/Goods/109303143


## (2) 범위내의 페이지 정보를 데이타프레임으로 출력하는 함수 생성후 확인하기 

In [12]:
def book_print(start=1, end=1):
    
    base_url = 'http://yes24.com'
    book_list = []
    rank = 0
    
    for i in range(start, end+1):
        # 주소 생성 
        url = 'http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=' + str(i)
        print(url)

        res = requests.get(url)
        html_str = res.text

        soup = BeautifulSoup(html_str, 'html.parser')
        td_list = soup.select('td.goodsTxtInfo')
#         len(td_list)


        for td in td_list:
            rank += 1
            title = td.select_one('a').text
            url = base_url + td.select_one('a').get('href')
            temp = td.select_one('div.aupu').text.split('| ')
            writer = temp[0].strip().split('\r\n')[0].strip()
            publisher = temp[1].strip()
            published_date = temp[2].strip()
            price = td.select('p')[1].text.split()[0]
            discount_price = td.select_one('span.priceB').text
            book_list.append([rank, title, writer, publisher, published_date, price, discount_price, url])
    
    
    
    df_book = pd.DataFrame(book_list, 
                       columns=['순위', '제목','저자','출판사','발행일', '가격', '할인가', 'URL'])    

    return df_book

# (3)전체 페이지 csv 파일로 저장하기 

In [13]:
base_url = 'http://yes24.com'
book_list = []
rank = 0
page = 0
    
while True:
    page += 1
    url = 'http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=' + str(page)
    print(url)

    res = requests.get(url)
    html_str = res.text

    soup = BeautifulSoup(html_str, 'html.parser')
    td_list = soup.select('td.goodsTxtInfo')
    print(len(td_list))

    # 책 정보가 있다면 
    if td_list: 
        for td in td_list:
            rank += 1
            title = td.select_one('a').text
            url = base_url + td.select_one('a').get('href')
            temp = td.select_one('div.aupu').text.split('| ')
            writer = temp[0].strip().split('\r\n')[0].strip()
            publisher = temp[1].strip()
            published_date = temp[2].strip()
            price = td.select('p')[1].text.split()[0]
            discount_price = td.select_one('span.priceB').text
            book_list.append([rank, title, writer, publisher, published_date, price, discount_price, url])
    else:
        break

print('데이타 저장 종료')

http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=1
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=2
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=3
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=4
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=5
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=6
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=7
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=8
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=9
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=10
20
http://www.yes24.com/24/Category/BestSeller?CategoryNumber=001&sumgb=06&PageNumber=11
20
http://www.yes24.com/24/Catego

In [14]:
df_book = pd.DataFrame(book_list, 
                       columns=['순위', '제목','저자','출판사','발행일', '가격', '할인가', 'URL'])    

In [15]:
df_book.shape

(997, 8)

In [16]:
df_book.head()

,순위,제목,저자,출판사,발행일,가격,할인가,URL
0,1,문재인의 위로,문재인 저,더휴먼,2022년 05월,"15,800원","14,220원",http://yes24.com/Product/Goods/109303129
1,2,5년 후 나에게 Q&A a day (2022 메리골드 Edition),포터 스타일 저/정지현 역,토네이도,2021년 11월,"16,800원","15,120원",http://yes24.com/Product/Goods/22793441
2,3,불편한 편의점,김호연 저,나무옆의자,2021년 04월,"14,000원","12,600원",http://yes24.com/Product/Goods/99308021
3,4,작별인사,김영하 저,복복서가,2022년 05월,"14,000원","12,600원",http://yes24.com/Product/Goods/108887930
4,5,대통령 문재인 명연설 100,편집부 편,더휴먼,2022년 05월,"22,000원","19,800원",http://yes24.com/Product/Goods/109303143
